# View light-Curves and Histograms for repeatability

- author Sylvie Dagoret-Campagne
- creation date 2024-09-03
- last update 2024-09-03
- last update : 2024-09-06 : histograms for different mag threshold
- last update : 2024-09-07 : add aperture flux in addition to psf flux by acting a flag and show error vs mag

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
path_photom ="data/photom/sources_objectTable"
tract = 3864

#inputfilename ="objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069-tract3864.csv"
inputfilename = "objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146-tract3864.csv"
inputfullfilename= os.path.join(path_photom,inputfilename) 

df_allsources = pd.read_csv(inputfullfilename,index_col=0)

In [ ]:
df_allsources

In [ ]:
df_allsources["apFlux_35_0_instMag"] = -2.5*np.log10(df_allsources["apFlux_35_0_instFlux"]) + df_allsources["zeroPoint"]
df_allsources["apFlux_35_0_instMagErr"] = 2.5/np.log(10.)*(df_allsources["apFlux_35_0_instFluxErr"]/df_allsources["apFlux_35_0_instFlux"]) 

In [ ]:
len(df_allsources.obj_index.unique())

In [ ]:
list_of_files = os.listdir(path_photom) 

In [ ]:
all_objnum = []
tag = "^sources_objectTable-t3864-multibands-o(.*)-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv$"
for idx,file in enumerate(list_of_files):
    coll = re.findall(tag,file)
    if len(coll)>0:
        all_objnum.append(int(coll[0]))
        
all_objnum = sorted(all_objnum)  

In [ ]:
all_df = []
for idx,idx_obj in enumerate(all_objnum):
    filename = f"sources_objectTable-t3864-multibands-o{idx_obj}-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv"
    fullfilename = os.path.join(path_photom,filename)
    df = pd.read_csv(fullfilename,index_col=0)
    df["apFlux_35_0_instMag"] = -2.5*np.log10(df["apFlux_35_0_instFlux"]) + df["zeroPoint"]
    df["apFlux_35_0_instMagErr"] = 2.5/np.log(10.)*(df["apFlux_35_0_instFluxErr"]/df["apFlux_35_0_instFlux"]) 
    all_df.append(df)
    

In [ ]:
df = all_df[0]

In [ ]:
df

In [ ]:
len(df_allsources[df_allsources.obj_index == all_objnum[0]] )

In [ ]:
residuals = {}
avgpsfmag = {}
residuals["g"] = [] 
residuals["r"] = [] 
residuals["i"] = [] 
residuals["z"] = [] 
residuals["y"] = [] 
avgpsfmag["g"] = []
avgpsfmag["r"] = []
avgpsfmag["i"] = []
avgpsfmag["z"] = []
avgpsfmag["y"] = []
FLAG_PSF = False
FLAG_CALIB = False

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 
colors = ["g","r","orange","grey","k"]

# loop on individual objects
for idx,df in enumerate(all_df):
    obj_num = df.obj_index.unique()[0]
    
    df_filt = {}
    
    # loop on filters
    
    for idx_b,b in enumerate(["g","r","i","z","y"]):
        df_filt[b] = df[df.band == b]
        
        
    fig,axs = plt.subplots(5,1,figsize=(12,8),sharex=True)
    
    axes_flat = axs.flatten()
    for idx_b,b in enumerate(["g","r","i","z","y"]):
        X = pd.to_datetime(df_filt[b]['obsStart'].values)
        
        
        if FLAG_PSF:    
            Y = df_filt[b]['psfMag'].values
            Yerr = df_filt[b]['psfMagErr'].values        
        else:
            if FLAG_CALIB:
                Y = df_filt[b]['apFlux_35_0_calMag'].values
                Yerr = df_filt[b]['apFlux_35_0_calMagErr'].values
            else:
                #Y = df_filt[b]['apFlux_35_0_instMag'].values
                #Yerr = df_filt[b]['apFlux_35_0_instMagErr'].values
                Y = df_filt[b]["apFlux_35_0_instMag"].values
                Yerr = df_filt[b]["apFlux_35_0_instMagErr"].values
                
                
                 
        
        N = len(Y)
        clippedMeanMagArray = scipy.stats.sigmaclip(Y, low=3.0, high=3.0).clipped
        clippedMeanMag = np.mean(clippedMeanMagArray)
        clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))
        clippeErrorMagArray = scipy.stats.sigmaclip(Y-clippedMeanMag,low=3.0, high=3.0).clipped
        clippeErrorMagSigma = (np.sqrt(np.sum(clippeErrorMagArray**2))/np.sqrt(len(clippeErrorMagArray)))*1000.

        if b in ["g","r","z","y"]:
                avgpsfmag[b].append(clippedMeanMag)
                residuals[b].append(clippeErrorMagArray)
              
        
        label = f" band {b} : mag = {clippedMeanMag:.1f} mag , rms = {clippeErrorMagSigma:.0f} mmag, N= {N}"
        axes_flat[idx_b].errorbar(X,(Y-clippedMeanMag)*1000,yerr=Yerr*1000,fmt='.',lw=0.5,c=colors[idx_b],ecolor=colors[idx_b]) 
        axes_flat[idx_b].scatter(X,Y-clippedMeanMag,marker='o',s=10,c=colors[idx_b],label=label) 
        axes_flat[idx_b].xaxis.set_major_locator(month_locator) # Locator for major axis only.
        axes_flat[idx_b].xaxis.set_minor_locator(day_locator)
        axes_flat[idx_b].xaxis.set_major_formatter(year_month_formatter)
        axes_flat[idx_b].set_xticks(axes_flat[idx_b].get_xticks(), axes_flat[idx_b].get_xticklabels(), rotation=45, ha='right')
        axes_flat[idx_b].grid()
        if idx_b == 0:
            if FLAG_PSF:  
                axes_flat[idx_b].set_title(f"psf Magnitude for object {obj_num}")
            else:
                if FLAG_CALIB:
                    axes_flat[idx_b].set_title(f"Aperture Magnitude (calib) for object {obj_num}")
                else:
                    axes_flat[idx_b].set_title(f"Aperture Magnitude (instrum) for object {obj_num}")
                    
                
        elif idx_b == 4:
            axes_flat[idx_b].set_xlabel("date obs") 
        axes_flat[idx_b].set_ylabel("dev (mmag)") 
        axes_flat[idx_b].legend(loc="upper left")
        axes_flat[idx_b].set_ylim(-50,50)
    plt.tight_layout()
    plt.show()
    
  
    
    

## Histograms

In [ ]:
band = "g"

In [ ]:
color = ["orange","k","g","b","r"]
thresholds = np.array([13.,14.,15.,16.,17.])
histoList1 = []
histoList2 = []
histoList3 = []
histoList4 = []
histoList5 = []
histoLists = [ histoList1, histoList2, histoList3, histoList4,histoList5]
Nthr = len(thresholds)

In [ ]:
the_list_of_LC_average = avgpsfmag[band]
the_list_of_LC_residuals = residuals[band]
NLC = len(the_list_of_LC_average)

## Loop on threshold
for ith,th in enumerate(thresholds):
    for iobs in range(NLC):
        mag = the_list_of_LC_average[iobs]
        res = the_list_of_LC_residuals[iobs] 
        if mag < th:
            histoLists[ith].append(res)

In [ ]:
for ith,th in enumerate(thresholds):
    print(ith,th,len(histoLists[ith]))
    histoLists[ith] = np.hstack(histoLists[ith])
    print(ith,th,len(histoLists[ith]))
    histoLists[ith]*=1000.  # convert in mmag


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5))
for ith,th in enumerate(thresholds):
    data = histoLists[4-ith]
    sigma = np.std(data)
    m = thresholds[4-ith]
    label = f" m < {m:.0f} mag\n$\sigma$ = {sigma:.1f} mmag"
    ax.hist(data,bins=50,range=(-100,100),facecolor=color[4-ith],alpha=0.9,label=label)
ax.legend(loc="upper left")
if FLAG_PSF:
    ax.set_title(f"Light curve residuals in band {band} (PSF Flux)")
else:
    if FLAG_CALIB:
        ax.set_title(f"Light curve residuals in band {band} (calib Aperture Flux)")
    else:
        ax.set_title(f"Light curve residuals in band {band} (instrum Aperture Flux)")
        
ax.set_xlabel("residuals (mmag)")

## error vs mag

In [ ]:
df_allsources.head() 

In [ ]:
df_allsources["psfMagErr_mmag"] =df_allsources["psfMagErr"]*1000.
df_allsources["apFlux_35_0_calMagErr_mmag"] =df_allsources["apFlux_35_0_calMagErr"]*1000.
df_allsources["apFlux_35_0_instMagErr_mmag"] =df_allsources["apFlux_35_0_instMagErr"]*1000.

df_allsources_g = df_allsources[df_allsources.band == "g" ]
df_allsources_r = df_allsources[df_allsources.band == "r" ]
df_allsources_i = df_allsources[df_allsources.band == "i" ]
df_allsources_z = df_allsources[df_allsources.band == "z" ]
df_allsources_y = df_allsources[df_allsources.band == "y" ]

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12,8))

ax1,ax2,ax3,ax4 = axs.flatten()

df_allsources_g.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="psfMag", y="psfMagErr_mmag",ax=ax1,marker=".",color="k",alpha=0.1)
ax1.set_title("PSF magnitude (calibrated)")

df_allsources_g.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax2,marker=".",color="k",alpha=0.1)
ax2.set_title("Aperture magnitude (calibrated with localZeroPoint)")

df_allsources_g.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="apFlux_35_0_instMag", y="apFlux_35_0_instMagErr_mmag",ax=ax3,marker=".",color="k",alpha=0.1)
ax3.set_title("Aperture instrumental magnitude (scaled with CCD ZeroPoint)")

ax4.set_axis_off()


ax1.legend()
ax1.set_xlim(11.,17)
ax1.set_ylim(0.,20.)
ax2.legend()
ax2.set_xlim(11.,17)
ax2.set_ylim(0.,20.)
ax3.legend()
ax3.set_xlim(11.,17)
ax3.set_ylim(0.,20.)


# generate legend
import matplotlib.patches as mpatches
green_patch = mpatches.Patch(color='green', label='g band',alpha=0.5)
red_patch = mpatches.Patch(color='red', label='r band',alpha=0.5)
orange_patch = mpatches.Patch(color='orange', label='i band',alpha=0.5)
grey_patch = mpatches.Patch(color='grey', label='z band',alpha=0.5)
black_patch = mpatches.Patch(color='k', label='y band',alpha=0.5)
ax1.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])
ax2.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])
ax3.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])



plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))

df_allsources_g.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="g",alpha=0.1)
df_allsources_r.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="r",alpha=0.1)
df_allsources_i.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="orange",alpha=0.1)
df_allsources_z.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="grey",alpha=0.1)
df_allsources_y.plot.scatter(x="apFlux_35_0_calMag", y="apFlux_35_0_calMagErr_mmag",ax=ax,marker=".",color="k",alpha=0.1)
ax.set_title("Aperture magnitude (Auxtel - SDSS' filters)")
ax.set_xlabel(" Magnitude (mag)")
ax.set_ylabel(" Magnitude error (mmag)")

# generate legend
import matplotlib.patches as mpatches
green_patch = mpatches.Patch(color='green', label='g band',alpha=0.5)
red_patch = mpatches.Patch(color='red', label='r band',alpha=0.5)
orange_patch = mpatches.Patch(color='orange', label='i band',alpha=0.5)
grey_patch = mpatches.Patch(color='grey', label='z band',alpha=0.5)
black_patch = mpatches.Patch(color='k', label='y band',alpha=0.5)
ax.legend(handles=[green_patch,red_patch, orange_patch, grey_patch,black_patch])